In [119]:
import pandas as pd
import numpy as np

In [2]:
metadata = pd.read_csv('all_metadata.csv')
metadata.head()

,name,composer,genre,key,year,instruments,details_url,tags,catalogue_code,file_urls,raw_file_path,source_url,processed_file_path
0,"N6e. Chopin, Mazurka, op. 41 no. 2, mm. 1-8","Chopin, Frederic",NaN,a minor,NaN,NaN,http://kern.ccarh.org/cgi-bin/ksdata?location=...,NaN,NaN,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,data/files_kernscores/859c17427b612d62aac51ab1...,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,data/normalized/859c17427b612d62aac51ab14ad809...
1,"Mazurka in C-sharp Minor, Op. 63, No. 3","Chopin, Frederic",NaN,c# minor,1846,NaN,http://kern.ccarh.org/cgi-bin/ksdata?location=...,NaN,NaN,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,data/files_kernscores/c05aff259adc395acca7cb42...,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,data/normalized/c05aff259adc395acca7cb42cff8b9...
2,Fugue,"Bach, Johann Sebastian",Baroque,b minor,1742,NaN,http://kern.humdrum.org/cgi-bin/ksdata?file=wt...,NaN,BWV 893,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,data/files_kernscores/cd1f471606732412448ecafc...,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,data/normalized/cd1f471606732412448ecafc853a2b...
3,Piano Sonata No. 3 in B-flat major,"Mozart, Wolfgang Amadeus",NaN,B- major,1774,NaN,http://kern.humdrum.org/cgi-bin/ksdata?file=so...,NaN,K,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,data/files_kernscores/fa8b6eb1aa5e7a31156d2e9f...,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,data/normalized/fa8b6eb1aa5e7a31156d2e9fcf4721...
4,Fugue,"Bach, Johann Sebastian",Baroque,g# minor,1742,NaN,http://kern.humdrum.org/cgi-bin/ksdata?file=wt...,NaN,BWV 892,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,data/files_kernscores/5e5c35310383699098841f2a...,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,data/normalized/5e5c35310383699098841f2a7760a6...


In [3]:
len(metadata.index)

1196

In [4]:
# Carl Philipp Emanuel Bach
metadata.composer[metadata['composer'] == 'C. P. E. Bach'] = 'cpe_bach'

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
import re
from unidecode import unidecode

def remove_initials(string):
    return re.sub('\s*\w\.\s*', '', string)

def normalize_composer_name(composer_name):
    composer_name = remove_initials(composer_name)
    composer_name = unidecode(composer_name)
    composer_name = composer_name.lower()
    return composer_name.split(',')[0]

In [6]:
metadata['composer'] = metadata['composer'].map(normalize_composer_name)

In [7]:
composers = metadata['composer'].unique()

In [8]:
composer_name_regex = '|'.join(composers)
metadata['name'][metadata['name'].str.contains(composer_name_regex, case=False)]

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


0             N6e. Chopin, Mazurka, op. 41 no. 2, mm. 1-8
384     Reminiscences de Don Juan\n(Fantasia on Mozart...
419     53 Studies on Études on Frédéric Chopin\n(1914...
429     M. P. Heller: Joy of Christmas\n(Christmas Fan...
430     Gustav Lange: To Christmas\n(Fantasia)\n, Opus...
434     Ferd. Keller: Christmas Fantasia Christmas Fan...
435     Robert Leonard: Christmas Fantasia, Opus 99 Ch...
441     Lyric Pieces Book III, Opus 43\n(1886)\n No. 2...
487     The Seasons, Opus 37a\n(1876)\n December - Chr...
1025                             Fugue sur le nom de Bach
Name: name, dtype: object

In [9]:
def extract_year_from_title(title_string):
    match = re.search('\((\d{4})\)', title_string)
    if match:
        return int(match.group(1))
    
    return None

In [10]:
extract_year_from_title('Sonatina Opus 36 No. 1, Opus 36\n(1797)\n')

1797

In [190]:
#metadata['year'] = pd.read_csv('all_metadata.csv')['year']

In [12]:
years_from_title = metadata['name'].map(extract_year_from_title)

for index, row in metadata[years_from_title.notnull() & metadata['year'].notnull()].iterrows():
    raise Exception('Warning! Year present both in name ({0}) and in metadata ({1})'.format(row['name'], row['year']))

metadata['year'] = metadata['year'].fillna(years_from_title)

In [13]:
metadata.ix[0, 'year'] = '1838'
metadata.ix[21, 'year'] = '1738'
metadata.ix[28, 'year'] = '1817'
metadata.ix[29, 'year'] = '1816'
metadata.ix[30, 'year'] = '1816'
metadata.ix[31, 'year'] = '1817'
metadata.ix[32, 'year'] = '1801'
metadata.ix[38, 'year'] = '1801'
metadata.ix[39, 'year'] = '1801'
metadata.ix[40, 'year'] = '1788'
metadata.ix[41, 'year'] = '1800'

In [163]:
def extract_year(year_obj):
    
    if not year_obj or str(year_obj) == 'nan':
        return None, False
    
    try:
        int_year = int(year_obj)
        return int_year, True
    except Exception:
        pass
    
    try:
        int_year = math.floor(year_obj)
        return int_year, True
    except Exception:
        pass
    
    match = re.search('(~|c|around|c.)?\s*(\d{4})\??', year_obj) # year with ~ or ?
    if match:
        return int(match.groups()[1]), False
    
    match = re.search('(\d{4})-\d{1,4}', year_obj) # year period. Return lower bound
    if match:
        return int(match.groups()[0]), False
    
    match = re.search('.+-(\d{4})', year_obj) # year period. Return lower bound
    if match:
        return int(match.groups()[0]), False
    
    
    match = re.search('(\d{4})/\w{2,4}', year_obj) # exact date
    if match:
        return int(match.groups()[0]), True
    
    match = re.search('\w{3,4}\.*(\d{4})', year_obj) # month and year
    if match:
        return int(match.groups()[0]), True
    
    match = re.search('(\d{2})th', year_obj) # century
    if match:
        return int(match.groups()[0]) * 100 - 50, False
    
    return None, False

In [191]:
extracted_years = metadata['year'].map(extract_year)

# todo: there should be a better way...
metadata['year'] = pd.to_numeric(extracted_years.map(lambda x: x[0]))
metadata['year_exact'] = extracted_years.map(lambda x: x[1])

In [173]:
from collections import namedtuple

Period = namedtuple('Period', ['start', 'end'])

music_periods = { # this is not exactly exact. MUST DOUBLE CHECK AFTER
    'baroque': Period(1600, 1750), 
    'classicism': Period(1750, 1820), 
    'romanticism': Period(1820, 1890), 
    'modern': Period(1890, 1950),
    'folk': Period(1000, 1600),
    'jazz': Period(0, 0) # sorry jazz. i love you it is just not your day
}

metadata['period'] = metadata['genre'].map(lambda x: x.lower() if x else None)
metadata[~metadata['period'].isin(music_periods.keys())]['period'] = None
metadata[metadata['period'] == 'romantic']['genre'] = 'romanticism'

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [328]:
def get_form_from_title(title_string):
    if not title_string:
        return None
    
    title_string = unidecode(title_string)
    for form in musical_forms:
        if re.search(form, title_string, re.IGNORECASE):
            return form
    
    return None

In [331]:
musical_forms = [
    'rag',
    'sonata',
    'sonatina',
    'rhapsod',
    'suite',
    'canon',
    'nocturne',
    'valse',
    'ballad',
    'polonaise',
    'fantasia',
    'song', 
    'march', 
    'technique', 
    'etude', 
    'mazurka', 
    'fugue', 
    'rondo',
    'prelude']

metadata['musical_form'] = metadata['genre'].map(lambda x: get_form_from_title(str(x)))

In [333]:
metadata['musical_form'] = metadata['musical_form'].fillna(metadata['name'].map(get_form_from_title))

In [205]:
def get_period_by_year(year):
    if not year or year is None or str(year) == 'nan':
        return None
    
    for period_name, period_dates in music_periods.items():
        if period_dates.start <= year <= period_dates.end:
            return period_name
        
    return None

In [207]:
metadata['period'] = metadata['period'].fillna(metadata['year'].map(get_period_by_year))

In [221]:
# map map map over map
# see map map in the map
# map grabs map by map
# mapmapmap mapmap mapmap

period_by_composer = metadata.groupby(by=['composer'])['period'].unique()
period_by_composer = period_by_composer.map(lambda periods: [x for x in periods if x is not None])
period_by_composer = period_by_composer.map(lambda periods: (len(periods), periods))

definite_composers = period_by_composer[period_by_composer.map(lambda x: x[0]) == 1].map(lambda x: x[1])
print('\ndefinite_composers', definite_composers)

ambiguous_composers = period_by_composer[period_by_composer.map(lambda x: x[0]) > 1].map(lambda x: x[1])
print('\nambiguous_composers', ambiguous_composers)

unknown_composers = period_by_composer[period_by_composer.map(lambda x: x[0]) == 0].map(lambda x: x[1])
print('\nunknown_composers', unknown_composers)


definite_composers composer
albeniz            [romanticism]
andre               [classicism]
anonymous          [romanticism]
ascher                  [modern]
balakirew          [romanticism]
benda               [classicism]
bizet              [romanticism]
borodin            [romanticism]
brahms             [romanticism]
brown                   [modern]
burgmueller        [romanticism]
burgmuller         [romanticism]
chopin             [romanticism]
comeau                    [folk]
cpebach             [classicism]
doonan                    [jazz]
fischer                [baroque]
godowsky           [romanticism]
granados           [romanticism]
handel                 [baroque]
hanon              [romanticism]
haydn               [classicism]
hunter                    [jazz]
kruetzer           [romanticism]
kuhlau             [romanticism]
liszt              [romanticism]
mendelssohn        [romanticism]
moszkowski         [romanticism]
muller              [classicism]
mussorgsky    

In [232]:
definite_periods = metadata[metadata['composer'].isin(definite_composers.index)]['composer'].map(lambda x: definite_composers[x][0])
metadata['period'] = metadata['period'].fillna(definite_periods)

In [346]:
import os.path
metadata['filename'] = metadata['processed_file_path'].map(lambda x: os.path.basename(str(x)))

In [352]:
del metadata['processed_file_path']
del metadata['raw_file_path']

In [353]:
metadata.to_csv('cleaned_metadata.csv')

In [354]:
metadata.head()

,name,composer,key,year,instruments,details_url,tags,catalogue_code,file_urls,source_url,period,year_exact,musical_form,filename
0,"N6e. Chopin, Mazurka, op. 41 no. 2, mm. 1-8",chopin,a minor,NaN,NaN,http://kern.ccarh.org/cgi-bin/ksdata?location=...,NaN,NaN,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,romanticism,False,mazurka,859c17427b612d62aac51ab14ad809c7ff62d722.xml
1,"Mazurka in C-sharp Minor, Op. 63, No. 3",chopin,c# minor,1846,NaN,http://kern.ccarh.org/cgi-bin/ksdata?location=...,NaN,NaN,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,romanticism,True,mazurka,c05aff259adc395acca7cb42cff8b941f63dfb0a.xml
2,Fugue,bach,b minor,1742,NaN,http://kern.humdrum.org/cgi-bin/ksdata?file=wt...,NaN,BWV 893,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,baroque,True,fugue,cd1f471606732412448ecafc853a2b68da0c95f8.xml
3,Piano Sonata No. 3 in B-flat major,mozart,B- major,1774,NaN,http://kern.humdrum.org/cgi-bin/ksdata?file=so...,NaN,K,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,http://kern.humdrum.org/cgi-bin/ksdata?l=users...,classicism,True,sonata,fa8b6eb1aa5e7a31156d2e9fcf47218dc699e0eb.xml
4,Fugue,bach,g# minor,1742,NaN,http://kern.humdrum.org/cgi-bin/ksdata?file=wt...,NaN,BWV 892,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,http://kern.humdrum.org/cgi-bin/ksdata?l=mused...,baroque,True,fugue,5e5c35310383699098841f2a7760a6f6e0a09a36.xml
